# Train sktime-dl LSTM-FCN Model

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Only print warnings, ignore info and error 
os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # Disable GPU

# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' 
# import tensorflow as tf
# tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)

import time
import numpy as np
import pandas as pd

from tensorflow.keras import callbacks

from sktime.classification.deep_learning.mlstmfcn import MLSTMFCNClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

C:\Users\nicol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\nicol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sktime\utils\validation\_dependencies.py:117: UserWarning: No module named 'keras_self_attention'. 'keras-self-attention' is a soft dependency and not included in the base sktime installation. Please run: `pip install keras-self-attention` to install the keras-self-attention package. To install all soft dependencies, run: `pip install sktime[all_extras]`
  warnings.warn(msg)


In [3]:
# you can use a larger subset, if you like, there are:
# - SITS-train-phase2-subset-1000000.csv.gz
# - SITS-train-phase2-subset-100000.csv.gz
# - SITS-train-phase2-subset-10000.csv.gz
DATA_TRAIN = "./dataset/SITS-train-phase2-subset-1000000.csv.gz"
use_univariate = False

# Set univariate=False, if you use a classifier with multivariate capabilities
def read_data_sktime(DATA, univariate=False):
    data = pd.read_csv(DATA, delimiter="," , 
                       na_values=['?'], dtype='float', 
                       index_col="id", compression='gzip')

    # Fill NaN values
    # We use the most basic way with bfill and ffill to carry on the last values
    data.fillna(method='bfill', inplace=True, axis=1)
    data.fillna(method='ffill', inplace=True, axis=1)

    # Extract Data and Labels
    X = np.array(data.iloc[:,1:].values)
    y = np.array(data.iloc[:,0].astype(int))


    if univariate:
        X = X.reshape(X.shape[0], 1, X.shape[1])
    else:
        X = X.reshape(X.shape[0], 3, X.shape[1]//3)
        
        
    print(X.shape)
    return X, y

X, y = read_data_sktime(DATA_TRAIN, univariate=use_univariate)

for (train_ix, test_ix)  in StratifiedShuffleSplit(n_splits=1, test_size=0.05, random_state=42).split(X, y):
    X_train, y_train = X[train_ix], y[train_ix]
    X_val, y_val = X[test_ix], y[test_ix]


    print("Training data:")
    print(X_train.shape)
    print(y_train.shape)
    print()

    print("Validation data:")
    print(X_val.shape)
    print(y_val.shape)
    print()


(1000000, 3, 46)
Training data:
(950000, 3, 46)
(950000,)

Validation data:
(50000, 3, 46)
(50000,)



In [5]:
# Define callbacks

reduce_lr = callbacks.ReduceLROnPlateau(monitor="loss", factor=0.7, patience=10, min_lr=0.0001)
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
tensorboard = callbacks.TensorBoard(log_dir="./tensorboard/phase_2_mlstm_full_dataset_tuned_and_fixed_multivariate_with_dilation", histogram_freq=1)
# model_save = callbacks.ModelCheckpoint('best_model_exploring.h5', monitor='val_loss', mode='min', save_best_only=True)

callbacks_ = [early_stopping, reduce_lr, tensorboard]

In [4]:
class MLSTMFCNWithValidation(MLSTMFCNClassifier):

    def fit(self, X, y, X_val=None, y_val=None, use_dimension_shuffle = True,  **kwargs):
        self.reset()

        start = int(round(time.time() * 1000))
        # convenience conversions to allow user flexibility:
        # if X is 2D array, convert to 3D, if y is Series, convert to numpy
        X, y = self._internal_convert(X, y)
        X_metadata = self._check_classifier_input(X, y)
        missing = X_metadata["has_nans"]
        multivariate = not X_metadata["is_univariate"]
        unequal = not X_metadata["is_equal_length"]
        self._X_metadata = X_metadata

        # Check this classifier can handle characteristics
        self._check_capabilities(missing, multivariate, unequal)

        # remember class labels
        self.classes_ = np.unique(y)
        self.n_classes_ = self.classes_.shape[0]
        self._class_dictionary = {}
        for index, class_val in enumerate(self.classes_):
            self._class_dictionary[class_val] = index

        # escape early and do not fit if only one class label has been seen
        #   in this case, we later predict the single class label seen
        if len(self.classes_) == 1:
            self.fit_time_ = int(round(time.time() * 1000)) - start
            self._is_fitted = True
            return self

        # Convert data as dictated by the classifier tags
        X = self._convert_X(X)
        multithread = self.get_tag("capability:multithreading")
        if multithread:
            try:
                self._threads_to_use = check_n_jobs(self.n_jobs)
            except NameError:
                raise AttributeError(
                    "self.n_jobs must be set if capability:multithreading is True"
                )

        # pass coerced and checked data to inner _fit
        self._fit(X, y, X_val, y_val, use_dimension_shuffle, **kwargs)
        self.fit_time_ = int(round(time.time() * 1000)) - start

        # this should happen last
        self._is_fitted = True
        return self

    def _fit(self, X, y, X_val=None, y_val=None, use_dimension_shuffle=True, **kwargs):
        from sklearn.utils.validation import check_random_state
        
        self.random_state = check_random_state(self.random_state)
        y_onehot = self.convert_y_to_keras(y)
        
        if y_val is not None:
            y_val = self.label_encoder.transform(y_val)
            y_val = y_val.reshape(-1, 1)
            y_val = self.onehot_encoder.transform(y_val)

        if use_dimension_shuffle:
            #Transpose to conform to Keras input style.
            X = X.transpose(0, 2, 1)
            
            if X_val is not None:
                X_val = X_val.transpose(0, 2, 1)
             
        validation_data = (X_val, y_val) if X_val is not None and y_val is not None else None

        # ignore the number of instances, X.shape[0],
        # just want the shape of each instance
        self.input_shape = X.shape[1:]

        self.model_ = self.build_model(self.input_shape, self.n_classes_)

        if self.verbose:
            self.model_.summary()

        self.history = self.model_.fit(
            X,
            y_onehot,
            batch_size=self.batch_size,
            epochs=self.n_epochs,
            verbose=self.verbose,
            callbacks=self.callbacks,
            validation_data=validation_data,
            **kwargs
        )

        self._is_fitted = True

        return self
    
    def predict(self, X, use_dimension_shuffle=True) -> np.ndarray:
        self.check_is_fitted()

        # boilerplate input checks for predict-like methods
        X = self._check_convert_X_for_predict(X)

        # handle the single-class-label case
        if len(self._class_dictionary) == 1:
            return self._single_class_y_pred(X, method="predict")

        # call internal _predict_proba
        return self._predict(X, use_dimension_shuffle)
    
    def _predict(self, X, use_dimension_shuffle=True, **kwargs):
        probs = self._predict_proba(X, use_dimension_shuffle=True, **kwargs)
        
        from sklearn.utils import check_random_state
        rng = check_random_state(self.random_state)
        return np.array(
            [
                self.classes_[int(rng.choice(np.flatnonzero(prob == prob.max())))]
                for prob in probs
            ]
        )

    def _predict_proba(self, X, use_dimension_shuffle=True, **kwargs):
        
        if use_dimension_shuffle:
            # Transpose to work correctly with keras
            X = X.transpose((0, 2, 1))
        
        probs = self.model_.predict(X, self.batch_size, **kwargs)

        # check if binary classification
        if probs.shape[1] == 1:
            # first column is probability of class 0 and second is of class 1
            probs = np.hstack([1 - probs, probs])
        probs = probs / probs.sum(axis=1, keepdims=1)
        return probs    

In [81]:
clf = MLSTMFCNWithValidation(
    n_epochs=200,
    attention=False,
    batch_size= 32, 
    # dilation_rate= 2, 
    filter_sizes= (64, 128, 64),
    kernel_sizes= (5, 3, 1),
    lstm_size= 3,
    callbacks=callbacks_,
    verbose=True,
)

print(X_train.transpose().shape)
print(y_train.transpose().shape)
print(X_val.transpose().shape)
print(y_val.transpose().shape)

clf.fit(X_train, y_train, X_val, y_val, use_dimension_shuffle=False)

(46, 3, 950000)
(950000,)
(46, 3, 50000)
(50000,)
Model: "model_25"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_26 (InputLayer)          [(None, 3, 46)]      0           []                               
                                                                                                  
 conv1d_75 (Conv1D)             (None, 3, 128)       47232       ['input_26[0][0]']               
                                                                                                  
 batch_normalization_75 (BatchN  (None, 3, 128)      512         ['conv1d_75[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_75 (Activation)     (None, 3,

In [82]:
DATA_TEST = "./dataset/SITS-test-data-phase2-nolabel.csv.gz"

X_test, _ = read_data_sktime(DATA_TEST, univariate=use_univariate)
# X_test, _ = read_data_sktime(DATA_TEST, univariate=True)
print(X_test)

# Make a prediction
predictions = clf.predict(X_test, use_dimension_shuffle=False)


# Create a submission file for kaggle
submission = pd.DataFrame({'PREDICTED': predictions})
submission.index.name="ID"

filename = 'baseline_mlstm_submission_multivariate_new_params_phase2.csv'
submission.to_csv(filename,index=True)
print('Saved file: ' + filename)

#Visualize the first 5 rows
submission.head()

(20000, 3, 46)
[[[157. 175. 177. ... 293. 300.  90.]
  [ 90.  93.  88. ...  44.  43.  75.]
  [ 75.  77.  72. ...  51.  50.  50.]]

 [[241. 221. 246. ... 266. 263. 262.]
  [ 96.  99.  89. ...  66.  59.  60.]
  [ 76.  73.  80. ...  67.  67.  60.]]

 [[190. 178. 188. ... 214. 176.  73.]
  [ 73.  73.  76. ...  57.  48.  61.]
  [ 61.  58.  56. ...  44.  51.  51.]]

 ...

 [[106. 106. 140. ... 118. 101.  70.]
  [ 70.  70.  90. ...  78.  64.  59.]
  [ 59.  59.  72. ...  63.  53.  53.]]

 [[175. 175. 208. ... 179. 146. 175.]
  [ 93.  93. 102. ... 112.  77.  68.]
  [ 81.  81.  91. ...  95.  68.  69.]]

 [[208. 209. 211. ... 328. 283.  67.]
  [ 67.  56.  68. ...  51.  53.  55.]
  [ 55.  54.  58. ...  66.  68.  68.]]]
(20000, 3, 46)
625/625 [==============================] - 4s 4ms/step
Saved file: baseline_mlstm_submission_multivariate_new_params_phase2.csv


,PREDICTED
ID,
0,1
1,9
2,9
3,18
4,3
